<a href="https://colab.research.google.com/github/ElinorZhang0/ElinorZhang0/blob/main/Get_contact_sign_up_for_a_form.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up

In [ ]:
pip install hubspot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for hubspot: filename=hubspot-0.1.14.dev0-py3-none-any.whl size=10448 sha256=ceb876117be81647e4e2329b69dd5ff777171daa99db616488bd7bfd999c8231
  Stored in directory: /root/.cache/pip/wheels/4e/d4/13/170189127bef41d667e354462aee1940d03537ab0fc3c75ce6
Successfully built hubspot


In [ ]:
!pip install --upgrade hubspot-api-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 5.2 MB/s 


In [ ]:
from google.colab import auth
auth.authenticate_user()


In [ ]:
!pip install google-cloud-secret-manager

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 97 kB 2.3 MB/s 
     |████████████████████████████████| 46 kB 3.2 MB/s 
     |████████████████████████████████| 1.0 MB 12.1 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220527125636 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.1 which is incompatible.


In [ ]:
import pandas as pd
from pprint import pprint
import requests
import hubspot
import json
from hubspot.crm.contacts import ApiException
from hubspot.crm.deals import SimplePublicObjectInput, ApiException
from hubspot.crm.associations import BatchInputPublicObjectId

In [ ]:
from google.cloud import secretmanager

def access_secret_version(project_id, secret_id, version_id):
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"
    response = client.access_secret_version(request={"name": name})
    payload = response.payload.data.decode("UTF-8")
    return payload

HAPI_KEY = access_secret_version("vivid-kite-315522", "hapi_key", "1")
client = hubspot.Client.create(api_key = HAPI_KEY)

# check forms

In [ ]:
#get all contact vid

count = 100
has_more = True
offset = ''
contact_info = []

while has_more:
  url = f'https://api.hubapi.com/contacts/v1/lists/all/contacts/all?hapikey={HAPI_KEY}&count={count}&property=vid&vidOffset={offset}'
  headers = {}
  headers['Content-Type']= 'application/json'
  r = requests.get(url = url, headers = headers)
  contact_info.append(json.loads(r.text))
  
  if str(r) != '<Response [200]>':
    break
  has_more = json.loads(r.text)['has-more']
  offset = json.loads(r.text)['vid-offset']




In [ ]:
# extract vid
vid = []
#pprint(list_of_dict_values)

#exclude staff from Greg's office
office = [3623301,3576251,3525351,3525001,3484351,3463001,3448651,3448601,3448551,3445951,3376801,3376601,3368001,3356051,
 3335101,3322051,3275851,3221851,3213651,3198201,3266501,3166151,3156551,3156501,3190651,3141251,3139251,3120001,3111251,
 3146651,3116951,3105351,2818601,2856251,1973251,3116901,1966901,3146701,1949701,267851,252464,251812,251440,251428,250970,
 248964,244709,244387,241512,237477,235922,174580,87633,83196,78924,75100,56217,54178,33986,32888,27561,25575,5384]
count = 0
key= 'contacts'

for info in contact_info:
  if key in info:
    contacts = info['contacts']
    count +=1
  else:
    continue


  for contact in contacts:
    if contact['canonical-vid'] not in office:
      vid.append(contact['canonical-vid'])



In [ ]:
#len(vid)

In [ ]:
#check numbers of form submission

# number of unquie form submitted: num_unique_conversion_events

count = 0
id_has_unquie_form_submission=[]
key = 'properties'

for id in vid:
  count +=1
  if count % 100 == 0:
    print(count)

  url = "https://api.hubapi.com/contacts/v1/contact/vid/"+str(id)+"/profile?hapikey=" + HAPI_KEY + "&formSubmissionMode=all"
  headers = {}
  try:
    r = requests.get(url = url, headers = headers)
    response_txt = json.loads(r.text)
    if key in response_txt:
      contact_properties = response_txt['properties']
      # print(form_submissions)
    else:
      continue
    #check if the person have submitted a form
    if bool(response_txt['properties']['num_unique_conversion_events']) == False:
      continue
      # print(id, "no form")
    elif bool(response_txt['properties']['num_unique_conversion_events']) == True:
      number_of_unqiue_form = response_txt['properties']['num_unique_conversion_events']['value']
      #print(number_of_unqiue_form)
      if number_of_unqiue_form == '0':
        continue
      else: 
        id_has_unquie_form_submission.append(id)
        print(id,'append')
  
  except Exception as e:
    continue
  



  # pprint(response_txt['properties']['num_unique_conversion_events']['value'])
      # print(type(response_txt))

174128 append
100
174579 append
200
174821 append
175031 append
300
175256 append
400
500
175913 append
600
700
800
900
1000
178021 append
1100
1200
1300
1400
1500
1600
1700
1800
1900
182608 append
2000
2100
2200
2300
2400
2500
2600
184544 append
184590 append
184687 append
2700
185101 append
2800
2900
3000
3100
186938 append
3200
3300
187639 append
3400
3500
188136 append
188259 append
3600
188511 append
3700
3800
189101 append
3900
189738 append
189743 append
4000
189998 append
4100
4200
190712 append
190741 append
190838 append
4300
191058 append
4400
4500
4600
4700
191971 append
4800
4900
5000
5100
5200
5300
5400
5500
194053 append
5600
5700
5800
5900
195196 append
6000
6100
6200
195706 append
6300
195842 append
6400
195974 append
195976 append
195979 append
195995 append
6500
6600
196379 append
196553 append
196561 append
6700
196722 append
6800
196777 append
196912 append
196937 append
6900
7000
197229 append
7100
7200
197772 append
7300
7400
7500
198329 append
7600
198709 append

In [ ]:
#check Form submission by vid 
count = 0
id_has_form = []
key= 'form-submissions'
for id in id_has_unquie_form_submission:
  count +=1
  if count % 100 == 0:
    print(count)

  url = "https://api.hubapi.com/contacts/v1/contact/vid/"+str(id)+"/profile?hapikey=" + HAPI_KEY + "&formSubmissionMode=all"
  headers = {}
  try:
    r = requests.get(url = url, headers = headers)
    response_txt = json.loads(r.text)
    # pprint(response_txt)
    # print(type(response_txt))
    
    if key in response_txt:
      form_submissions = json.loads(r.text)['form-submissions']
      # print(form_submissions)
    else:
      continue
    #check if the person have submitted a form
    if bool(form_submissions) == False:
      continue
      # print(id, "no form")
    elif bool(form_submissions) == True:
      #get the form names
      for submission in form_submissions:
        title = submission['title']
        #exclude the 'Intern Application Form' and 'Congress Volunteer Form'
        if title == 'Intern Application Form' or title == 'Congress Volunteer Form' or title == 'Volunteer Form':
          continue
        else: 
          id_has_form.append(id)
          print(id,'append')
            
  except Exception as e:
    continue
    # pprint(r.text)
    # print(count,e.message, e.args)
    # raise Exception("Exception Raised")


174128 append
174821 append
175031 append
175256 append
175913 append
178021 append
182608 append
184544 append
184590 append
184687 append
185101 append
186938 append
187639 append
188136 append
188259 append
188511 append
189101 append
189738 append
189743 append
189998 append
190712 append
190741 append
190741 append
190838 append
191058 append
191971 append
194053 append
195196 append
195196 append
195706 append
195842 append
195974 append
195976 append
195979 append
195995 append
196379 append
196553 append
196722 append
196777 append
196777 append
196912 append
196937 append
197229 append
197772 append
198329 append
198709 append
198772 append
198999 append
199392 append
199527 append
199639 append
199938 append
199997 append
199997 append
199997 append
200258 append
200272 append
200507 append
200566 append
200802 append
201034 append
201501 append
201706 append
201942 append
201946 append
202114 append
202246 append
202246 append
202742 append
202863 append
203066 append
203353

In [ ]:
# info['contacts'][0]

{'addedAt': 1651023268512,
 'canonical-vid': 4783451,
 'form-submissions': [],
 'identity-profiles': [{'deleted-changed-timestamp': 0,
   'identities': [{'timestamp': 1651023268445,
     'type': 'LEAD_GUID',
     'value': '9806fda2-2ae4-490c-933c-402fa446fa1c'}],
   'saved-at-timestamp': 1651023268452,
   'vid': 4783451}],
 'is-contact': True,
 'merge-audits': [],
 'merged-vids': [],
 'portal-id': 8907454,
 'properties': {'lastmodifieddate': {'value': '1651339787032'}},
 'vid': 4783451}

In [ ]:
# len(id_has_form)

167

In [ ]:
# id_has_form

# Function

In [ ]:
def get_contact_name(id):
    '''Get the contact name of a contact given the id'''
    url = f"https://api.hubapi.com/contacts/v1/contact/vid/{id}/profile?hapikey={HAPI_KEY}&property=firstname&property=lastname&property=email"
    headers = {}
    r = requests.get(url = url, headers = headers)
    pprint(r.text)

    properties = json.loads(r.text)['properties']
    first_name = properties['firstname']['value'] if properties['firstname']['value'] != None else ''
    last_name = properties['lastname']['value'] if properties['lastname']['value'] != None else ''
    
    name = first_name+' '+last_name if first_name+' '+last_name != ' ' else properties['email']['value']
    return name

In [ ]:
def create_deal(contact_id):
    '''Create a deal given a contact id'''
    url = f'https://api.hubapi.com/deals/v1/deal?hapikey={HAPI_KEY}'
    headers={}
    headers["Content-Type"]="application/json"

    name = get_contact_name(id)
    data = json.dumps({
    "associations": {
        "associatedVids": [
        contact_id
        ]
    },
    "properties": [
        {
        "value": name, 
        "name": "dealname"
        },
        {
        "value": "26201999", #26201999
        "name": "dealstage"
        },
        {
        "value": "default",
        "name": "pipeline"
        }
    ]
    })

    try:
        api_response = requests.post(url, headers = headers, data = data)
        pprint(api_response)
    except ApiException as e:
        print("Exception when calling basic_api->create: %s\n" % e)

In [ ]:
def check_deal_pre_stage(dealid):
    '''Returns True if current deal stage is to the left of clicked link, else return False'''
    api_response = client.crm.deals.basic_api.get_by_id(deal_id=dealid, archived=False)
    # print(api_response)
  # for deal in api_response.__dict__['_results']:
    properties = api_response.__dict__['_properties']
    if properties['dealstage'] in ['9545242', '9498025', '4730248', '4693304','4693305','4693306','4693307','26201999']: 
        return True
    else:
        return False

In [ ]:
def update_deal(dealid):
    '''Updates a deal to be in the responded to email deal stage'''
    url ='https://api.hubapi.com/deals/v1/deal/' + dealid + '?hapikey=' + HAPI_KEY
    headers={}
    headers["Content-Type"]="application/json"

    data = json.dumps({
    "properties": [
        {
        "value": "26201999",
        "name": "dealstage"
        }
    ]
    })

    try:
        post_response = requests.put(data=data, url=url, headers=headers)
        pprint(post_response)
    except ApiException as e:
        print("Exception when calling basic_api->create: %s\n" % e)

In [ ]:
def get_contact_deals(id):
    '''Get deals associated with a contact given the id'''
    client = hubspot.Client.create(api_key=HAPI_KEY)

    batch_input_public_object_id = BatchInputPublicObjectId(inputs=[{"id":str(id), "includePropertyVersion": True}])
    try:
        api_response = client.crm.associations.batch_api.read(from_object_type="contacts", to_object_type="deals", batch_input_public_object_id=batch_input_public_object_id)
        if api_response.__dict__['_num_errors'] == None:
            return [d.__dict__['_id'] for d in api_response.__dict__['_results'][0].__dict__['_to']]
        return None
    except ApiException as e:
        print("Exception when calling batch_api->read: %s\n" % e)

In [ ]:
# get_contact_deals('7118901')

['9203610800']

In [ ]:
# get_contact_deals(vid[1])

['8874082200']

In [ ]:
# check_deal_pre_stage('8874082200')

True

In [ ]:
# update_deal('4910409897')

<Response [200]>


In [ ]:
# create_deal(vid)


KeyError: ignored

In [ ]:
# For each contact, either create a new deal or update their deal
# dealid = '4693307'

count = 0
for id in id_has_form:
  # count +=1
  print(id)
  # if count == 1:
  #   continue
  contact_deals = get_contact_deals(id)
  print(contact_deals)


  if contact_deals == None:
    create_deal(id)

  else:
    for dealid in contact_deals:
      if check_deal_pre_stage(dealid):
        update_deal(dealid)


174128
['8008119404']
<Response [200]>
174821
['4910409897']
175031
['5995803162']
<Response [200]>
175256
['8822336612']
<Response [200]>
175913
['8823472256']
<Response [200]>
178021
['6686903206']
<Response [200]>
182608
['5995263283']
<Response [200]>
184544
['8822464526']
<Response [200]>
184590
['8822469393']
<Response [200]>
184687
['8569619525']
<Response [200]>
185101
['8822505010']
<Response [200]>
186938
['8822464530']
<Response [200]>
187639
['8823421730']
<Response [200]>
188136
['8874130475']
188259
['8823411613']
<Response [200]>
188511
['8822434109']
<Response [200]>
189101
['8768296576']
189738
['8822486283']
<Response [200]>
189743
['8822505013']
<Response [200]>
189998
['8823448228']
<Response [200]>
190712
['8822495044']
<Response [200]>
190741
['8822469405']
<Response [200]>
190741
['8822469405']
<Response [200]>
190838
['8823438249']
<Response [200]>
191058
['8571163412']
<Response [200]>
191971
['8823448231']
<Response [200]>
194053
['8571320901']
<Response [200]

## Code not used in this project

In [ ]:
import requests
import json

url = f'https://api.hubapi.com/deals/v1/deal/9203610800?hapikey={HAPI_KEY}&includePropertyVersions=true'

r= requests.get(url = url);

pprint(r.text)

('{"portalId":8907454,"dealId":9203610800,"isDeleted":false,"associations":{"associatedVids":[7118901],"associatedCompanyIds":[],"associatedDealIds":[],"associatedTicketIds":[]},"properties":{"hs_closed_amount_in_home_currency":{"value":"0","timestamp":1655348198091,"source":"CALCULATED","sourceId":"CalculatedPropertyComputer","updatedByUserId":null,"versions":[{"name":"hs_closed_amount_in_home_currency","value":"0","timestamp":1655348198091,"sourceId":"CalculatedPropertyComputer","source":"CALCULATED","sourceVid":[]}]},"hs_analytics_latest_source_contact":{"value":"OFFLINE","timestamp":1655348198783,"source":"CALCULATED","sourceId":"RollupProperties","updatedByUserId":null,"versions":[{"name":"hs_analytics_latest_source_contact","value":"OFFLINE","timestamp":1655348198783,"sourceId":"RollupProperties","source":"CALCULATED","sourceVid":[]}]},"dealname":{"value":"MOVE '
 'TEMP TEST '
 'DEAL","timestamp":1655348198091,"source":"CRM_UI","sourceId":null,"updatedByUserId":25366716,"versions

In [ ]:
#get Form submission mode by Get a contact by vid 
import requests
import json
vid = ['3128351', '3626501','3615101']

for id in vid:
  url = "https://api.hubapi.com/contacts/v1/contact/vid/"+id+"/profile?hapikey=" + HAPI_KEY + "&formSubmissionMode=all"
  headers = {}
  r = requests.get(url = url, headers = headers)
  pprint(json.loads(r.text))


In [ ]:
# extract form name 
form_submissions = json.loads(r.text)['form-submissions']

for submission in form_submissions:
  title = submission['title']
pprint(title)


'Congress Volunteer Form'


In [ ]:
# check if contact submit a form by vid (exclude 'Intern Application Form' & 'Congress Volunteer Form')
if title == 'Intern Application Form':
    #print(vid + 'y')
    print('IAF')
elif title == 'Congress Volunteer Form':
    print('CVF')
else:
  print(vid + ": " + "TRUE")


CVF


In [ ]:
#get all form (non-marketing forms are filtered out)

import requests
import json


url = "https://api.hubapi.com/forms/v2/forms?hapikey=" + HAPI_KEY + "&limit=1"

headers = {}

r = requests.get(url = url, headers = headers)
pprint(json.loads(r.text))

In [ ]:
#get form information 
import requests
import json

#url = "https://api.hubapi.com/contacts/v1/contact/vid/202246/profile?hapikey=" + HAPI_KEY + "&formSubmissionMode=all"
url = "https://api.hubapi.com/contacts/v1/contact/vid/202246/profile?hapikey=" + HAPI_KEY + "&formSubmissionMode=newest"
# url = "https://api.hubapi.com/contacts/v1/contact/vid/202246/&formSubmissionMode=all/profile?hapikey=" + HAPI_KEY 
headers = {}

r = requests.get(url = url, headers = headers)
pprint(json.loads(r.text))

      

In [ ]:
# Get submissions for a form
import requests
import json


#url = "https://api.hubapi.com/form-integrations/v1/submissions/forms/"+ "be52b37f-8a44-4a23-a663-919709924276"+ "?hapikey=" + HAPI_KEY 

#Get a single field from a form
#url = "https://api.hubapi.com/forms/v2/fields/be52b37f-8a44-4a23-a663-919709924276/firstname" + "?hapikey=" + HAPI_KEY 

#Get all fields from a form
url = "https://api.hubapi.com/forms/v2/fields/be52b37f-8a44-4a23-a663-919709924276" + "?hapikey=" + HAPI_KEY 

headers = {}
r = requests.get(url = url, headers = headers)

pprint(json.loads(r.text))

#change the deal-stage 


In [ ]:
id = json.loads(r.text)[1]['defaultValue']
print(id) 

Anderson


In [ ]:
# Get contacts in Hubspot
hapikey = HAPI_KEY 
count = 1
api_response = []
property_list = ['contact']
get_all_contacts_url = "https://api.hubapi.com/contacts/v1/lists/all/contacts/all?"
parameter_dict = {'hapikey': hapikey, 'count': count}
headers = {}

In [ ]:
import requests
import json

contact = "3557601"

#for contact_id in contact:
url = "https://api.hubapi.com/contacts/v1/contact/vid/"+contact+"/profile?hapikey=" + HAPI_KEY

headers = {}

r = requests.get(url = url, headers = headers)
pprint(r.text)